<a href="https://colab.research.google.com/github/tsm-mehmetakiftasoz/tsm_makif/blob/main/%D0%B3%D0%BE%D1%82%D0%BE%D0%B2%D0%BE_mb51_zayaviteli_excels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install xlsxwriter

In [2]:
import pandas as pd
import numpy as np
from google.colab import files

import os
import zipfile

In [3]:
df_mb51 = pd.read_csv('/content/MB51_base_CSV.csv', sep=None, engine='python')

In [4]:
df_mb51['Общая стоимость остатков'] = df_mb51['Общая стоимость остатков'].astype(str).str.replace(',', '.', regex=False).astype(float).astype(int)

In [5]:
#df_mb51.columns

In [6]:
result = df_mb51.groupby('Заявитель').agg({
    'Дирекция': 'first',  # İlk değeri al
    '﻿Завод': 'count',     # Satır sayısını al (metin olsa da sayar)
    'Общая стоимость остатков': 'sum'  # Toplam maliyet
}).reset_index()

In [7]:
#filtered_result = result[result['Заявитель'] == 'Pevtsov A.']

In [8]:
#df_mb51.dtypes

In [9]:
#burada tek kişi için yapılması istenirse yazılacak
df_mb51_Pevtsova = df_mb51[df_mb51['Заявитель'] == 'Pevtsov A.'].copy()

columns_order = [
    '﻿Завод',
    'Склад',
    'Обозначение склада',
    'Партия',
    'Номер материала',
    'RU Tanım',
    'TR Tanım',
    'На КонтрКачества',
    'СвобИспользЗапас',
    'Базовая ЕИ',
    'Ст-ть/своб. испол.',
    'Валюта',
    'Имя 1',
    'Вид материала',
    'Группа материалов',
    'Единица измерения веса',
    'Ст-ть/контр.кач-ва',
    'Общая стоимость остатков',
    'Краткий текст материала',
    'Проект/не проект',
    'Заказ на поставку',
    'Поставщик',
    'Наименование поставщика',
    'Заявитель',
    'Инвентарный номер',
    'Группа закупок',
    'Наименование группы закупок',
    'Наш знак',
    '№ Позиции ZMB52',
    'Дата ввода ZMB51',
    'Продолжительность',
    'Дирекция',
    'Справочник'
]

df_mb51_Pevtsova = df_mb51_Pevtsova[columns_order]


In [10]:
#df_mb51_Pevtsov.to_excel("df_mb51_Pevtsov.xlsx", index=False, engine="xlsxwriter")
#files.download("df_mb51_Pevtsov.xlsx")

In [11]:
#her bir zayaviteli için tek tek kaydetme
# 1. Gerekli sütun sırasını tanımla
columns_order = [
    '﻿Завод', 'Склад', 'Обозначение склада', 'Партия', 'Номер материала',
    'RU Tanım', 'TR Tanım', 'На КонтрКачества', 'СвобИспользЗапас', 'Базовая ЕИ',
    'Ст-ть/своб. испол.', 'Валюта', 'Имя 1', 'Вид материала', 'Группа материалов',
    'Единица измерения веса', 'Ст-ть/контр.кач-ва', 'Общая стоимость остатков',
    'Краткий текст материала', 'Проект/не проект', 'Заказ на поставку', 'Поставщик',
    'Наименование поставщика', 'Заявитель', 'Инвентарный номер', 'Группа закупок',
    'Наименование группы закупок', 'Наш знак', '№ Позиции ZMB52', 'Дата ввода ZMB51',
    'Продолжительность', 'Дирекция', 'Справочник'
]

# 2. Benzersiz Заявитель değerlerini al
zayaviteli_list = df_mb51['Заявитель'].dropna().unique()

# 3. Çıktıları kaydetmek için klasör oluştur (varsa geç)
os.makedirs("zayaviteli_excels", exist_ok=True)

# 4. Her Заявитель için döngü
for z_name in zayaviteli_list:
    # Dosya adı için geçersiz karakterleri temizle
    safe_name = "".join(c if c.isalnum() or c in "._-" else "_" for c in z_name)

    # Filtreleme ve sütun sıralama
    df_filtered = df_mb51[df_mb51['Заявитель'] == z_name].copy()

    # Sütun kontrolü: eksik olanları atla (varsa uyarı verir)
    missing_cols = [col for col in columns_order if col not in df_filtered.columns]
    if missing_cols:
        print(f"{z_name} için eksik sütun(lar): {missing_cols}")
        continue

    df_filtered = df_filtered[columns_order]

    # Excel olarak kaydet
    output_path = f"zayaviteli_excels/df_mb51_{safe_name}.xlsx"
    df_filtered.to_excel(output_path, index=False)

print("✅ Tüm dosyalar 'zayaviteli_excels/' klasörüne kaydedildi.")

✅ Tüm dosyalar 'zayaviteli_excels/' klasörüne kaydedildi.


In [12]:
zip_filename = "zayaviteli_excels.zip"

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk("zayaviteli_excels"):
        for file in files:
            if file.endswith(".xlsx"):
                file_path = os.path.join(root, file)
                zipf.write(file_path)

In [13]:
from google.colab import files
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>